In [1]:
from io import StringIO
from app.wadray import Wad, Ray, _W, _R
from app.utils import load_config, WEEK, DAY, parse_period

YAML_SETUP = """
module: app.prototype
risk_modules:
  - name: Roulette
    mcr_percentage: 100
    premium_share: 0
    ensuro_share: 0
  - name: Flight-Insurance
    mcr_percentage: 90
    premium_share: 3
    ensuro_share: 1.5
  - name: Fire-Insurance
    mcr_percentage: 80
    premium_share: 0
    ensuro_share: 0.5
etokens:
  - name: eUSD1WEEK
    expiration_period: 604800
  - name: eUSD1MONTH
    expiration_period: 2592000
  - name: eUSD1YEAR
    expiration_period: 31536000
"""

protocol = load_config(StringIO(YAML_SETUP))

In [2]:
print("LP1 deposits 1000, balance: {}".format(protocol.deposit("eUSD1YEAR", "LP1", _W(1000))))
protocol.fast_forward_time(WEEK)

eUSD1YEAR = protocol.etokens["eUSD1YEAR"]
eUSD1MONTH = protocol.etokens["eUSD1MONTH"]

print("After a week balance is the same: {}".format(eUSD1YEAR.balance_of("LP1")))


LP1 deposits 1000, balance: 1000
After a week balance is the same: 1000


We now create a new policy. The relevant parameters for policies are: 
- **payout**: the amount to be paid to the customer in case the policy is triggered 
- **premium**: the amount paid by the customer
- **loss_prob**: the probability (0..1) of the policy being triggered. This value is calculated (or estimated) by the risk module.
- **expiration**: when the policy expires (in seconds)

As a consecuence of these input parameters, other values are calculated:
- **pure_premium**: the part of the premium that should be enought to cover the average losses (if loss_prob is exact). Calculated as `payout * loss_prob`
- **mcr**: the amount that needs to be locked to cover the policy until it expires. It's calculated as `(payout-premium) * mcr_percentage`. The mcr_percentage is a parameter of the risk modules depending on the correlation between the policies (also regulatory requirements).  
- **interest_rate**: after substracting the pure_premium, there should be an excess of the premium that is the source of earnings for the LPs (after taking also the share for Ensuro and the risk module developer). That excess amount, in relation with the MCR and the policy duration and normalized to a yearly rate is the *interest rate* of a policy.



In [3]:
policy_1 = policy = protocol.new_policy(
    "Roulette", payout=_W(36), premium=_W(1), 
    loss_prob=_R(1/37), expiration=protocol.now() + WEEK
)

print(f"New Policy Created - MCR: {policy.mcr} / Pure Premium: {policy.pure_premium} / Interest Rate: {policy.interest_rate}")

for etoken, amount in policy.locked_funds:
    print(f"    Locked fund - Token: {etoken} = {amount}")

New Policy Created - MCR: 35 / Pure Premium: 0.972972972972973094 / Interest Rate: 0.040264754550468655
    Locked fund - Token: eUSD1YEAR = 35


After one day, LP1's balance changes because it gets *interest* from active policy.

If a new LP gets into the pool, she will get the interests from that day (proportionally to it's share of the pool), but not the previous ones.

In [4]:
protocol.fast_forward_time(DAY)
print(f"After a day LP1's balance: {eUSD1YEAR.balance_of('LP1')}")
print("LP2 deposits 2000, balance: {}".format(protocol.deposit("eUSD1YEAR", "LP2", _W(2000))))
protocol.fast_forward_time(DAY)
print(f"LP1's day 2 balance: {eUSD1YEAR.balance_of('LP1')}")
print(f"LP2's day 2 balance: {eUSD1YEAR.balance_of('LP2')}")


After a day LP1's balance: 1000.003861003861003843
LP2 deposits 2000, balance: 2000
LP1's day 2 balance: 1000.005148005148005124
LP2's day 2 balance: 2000.002573992635807057


Other or the same LPs can put money in the other pools with different expiration periods. The policies will be backed by all the eligible pools (based in the expiration time now, perhaps other criteria in the future) and the MCR will be distributed proportionally to the capital availability.

If we create a new policy with expiration in 10 days, only capital from pools eUSD1MONTH and eUSD1YEAR will be locked.

In [5]:
print("LP3 deposits 500 on eUSD1WEEK, balance: {}".format(protocol.deposit("eUSD1WEEK", "LP3", _W(500))))
print("LP3 deposits 1500 on eUSD1MONTH, balance: {}".format(protocol.deposit("eUSD1MONTH", "LP3", _W(1500))))

policy_2 = policy = protocol.new_policy(
    "Roulette", payout=_W(72), premium=_W(2), 
    loss_prob=_R(1/37), expiration=protocol.now() + 10 * DAY
)

print(f"New Policy Created - MCR: {policy.mcr} / Pure Premium: {policy.pure_premium} / Interest Rate: {policy.interest_rate}")

for etoken, amount in policy.locked_funds:
    print(f"    Locked fund - Token: {etoken} = {amount}")

LP3 deposits 500 on eUSD1WEEK, balance: 500
LP3 deposits 1500 on eUSD1MONTH, balance: 1500
New Policy Created - MCR: 70 / Pure Premium: 1.945945945945946189 / Interest Rate: 0.028185328185328058
    Locked fund - Token: eUSD1MONTH = 23.516156061841567086
    Locked fund - Token: eUSD1YEAR = 46.483843938158432914


Finally the policies are resolved. If policy is triggered (in favor of the customer), the money of the accrued interests is taken from the supporting tokens along with the required MCR to do the payout.

If policy not triggered, the premium amount will be added to the pool, substracting the accrued interest and the shares of the premium for Ensuro and the Risk Module.

In [6]:
protocol.fast_forward_time(DAY)
print("Before 1st policy triggered:")
for LP in ["LP1", "LP2"]:
    print(f"  {LP} - eUSD1YEAR Balance: {eUSD1YEAR.balance_of(LP)}")

protocol.resolve_policy("Roulette", policy_1.id, customer_won=True)
print("After 1st policy triggered (customer_won=True):")
for LP in ["LP1", "LP2"]:
    print(f"  {LP} - eUSD1YEAR Balance: {eUSD1YEAR.balance_of(LP)}")


protocol.fast_forward_time(2 * DAY)

print("Before 2nd policy triggered:")
for LP, etoken in [("LP1", eUSD1YEAR), ("LP2", eUSD1YEAR), ("LP3", eUSD1MONTH)]:
    print(f"  {LP} - {etoken.name} Balance: {etoken.balance_of(LP)}")

protocol.resolve_policy("Roulette", policy_2.id, customer_won=False)

print("After 2nd policy triggered (customer_won=False):")
for LP, etoken in [("LP1", eUSD1YEAR), ("LP2", eUSD1YEAR), ("LP3", eUSD1MONTH)]:
    print(f"  {LP} - {etoken.name} Balance: {etoken.balance_of(LP)}")

Before 1st policy triggered:
  LP1 - eUSD1YEAR Balance: 1000.007631501774469938
  LP2 - eUSD1YEAR Balance: 2000.007540966711230601
After 1st policy triggered (customer_won=True):
  LP1 - eUSD1YEAR Balance: 988.337103831246799428
  LP2 - eUSD1YEAR Balance: 1976.666575745212729343
Before 2nd policy triggered:
  LP1 - eUSD1YEAR Balance: 988.339496821925726492
  LP2 - eUSD1YEAR Balance: 1976.671361708091962316
  LP3 - eUSD1MONTH Balance: 1500.005447758161044353
After 2nd policy triggered (customer_won=False):
  LP1 - eUSD1YEAR Balance: 988.77861061150884478
  LP2 - eUSD1YEAR Balance: 1977.549585896431216888
  LP3 - eUSD1MONTH Balance: 1500.671890173195473343
